# Better Way 31. 재사용 가능한 @property 메서드에는 Descriptor를 사용하자

* `@property`의 문제점: 재사용성
  * Better Way 29.
  * `@property`로 데코레이트 하는 메서드를 같은 클래스에 속한 여러 속성에 사용하지 못하고,
  * 관련 없는 클래스에서도 재사용 할 수 없다.

In [1]:
class Homework(object):
    def __init__(self):
        self._grade = 0
        
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value

In [2]:
galileo = Homework()
galileo.grade = 95

* 학생들의 시험 성적을 매기는 예제
  * 시험은 여러 과목으로 구성되어 있음
  * 과목 별로 점수가 있음

In [3]:
class Exam(object):
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
        
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
            
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_value = value
        
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_value = value

* 코드가 장황해진다
  * 시험 영역마다 `@property`와 `_check_grade` 검증이 필요
  * 과제와 시험 이외의 항복에도 재사용하고 싶을때에도 마찬가지

### Descriptor
* Descriptor protocol: 속성에 대한 접근을 언어에서 해석할 방법을 정의한다.
* Descriptor class: 반복 코드 없이도 성적 검증 동작을 재사용할 수 있게 해주는 `__get__` `__set__` 메서드를 제공해준다
* 이런 목적으로는 Mixin 보다 좋은 방법
  * 한 클래스의 서로 다른 많은 속성에 같은 로직을 재사용할 수 있음

In [4]:
class Grade(object):
    def __get__(*args, **kwargs):
        # ...
        return 'spam'
    
    def __set__(*args, **kwargs):
        # ...
        pass
    
class Exam(object):
    # 클래스 속성
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

* 아직 제대로 구현되지 않은 코드
* 다음과 같이 프로퍼티를 할당한다고 하자.

In [5]:
exam = Exam()
exam.writing_grade = 40

* 위 코드는 다음과 같이 해석된다.

In [6]:
Exam.__dict__['writing_grade'].__set__(exam, 40)

* 프로퍼티를 얻어온다고 하자.

In [7]:
print(exam.writing_grade)

spam


* 위 코드는 다음과 같이 해석된다.

In [8]:
print(Exam.__dict__['writing_grade'].__get__(exam, Exam))

spam


In [9]:
# 추가 설명: Exam.__dict__ 안에는 클래스 속성이 포함된다.

class Test:
    spam = 10
    
    def __init__(self):
        egg = 15
        
Test.__dict__.keys()

dict_keys(['__module__', 'spam', '__init__', '__dict__', '__weakref__', '__doc__'])

* 동작 메커니즘
  * object 의 `__getattribute__` 메서드
    * Better Way 32
  * Exam 인스턴스에 writing_grade 속성이 없으면 대신 Exam 클래스의 속성을 따른다. 
    * '`__get__`, `__set__` 메서드를 갖추고 있는가? 그렇다면 디스크립터 프로토콜을 따르고 있군.'

* 다시한번 그럴듯하게 구현해보자.

In [10]:
class Grade(object):
    def __init__(self):
        self._value = 0
    
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value
        
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

* **불행히도**, 위의 코드는 잘못 구현되어 있어서 제대로 동작하지 않음
  * 한 Exam 인스턴스에 있는 여러 속성에 접근하는 것은 잘 동작하지만
  * 여러 Exam 인스턴스의 경우 기대하지 않은 동작을 한다.

In [11]:
# 하나의 Exam 인스턴스는 OK
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade= 99

print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


In [12]:
# 테스트: 검증도 원하는대로 동작함
first_exam.math_grade = 101

ValueError: Grade must be between 0 and 100

In [13]:
# 두번째 Exam 인스턴스를 만들었더니 아니 글쎄!
second_exam = Exam()
second_exam.writing_grade = 75

print('Second', second_exam.writing_grade, 'is right')
print('First ', first_exam.writing_grade, 'is wrong')

Second 75 is right
First  75 is wrong


* 한 Grade 인스턴스가 모든 Exam 인스턴스의 writing_grade 클래스 속성으로 공유되어 버린다.
  * `    writing_grade = Grade()` 이 부분
  * Exam 인스턴스를 생성할때마다 생성되는 게 아니라 Exam 클래스를 처음 정의할 때 한 번만 생성된다.
  * 이 문제를 해결하려면 각 Exam 인스턴스 별로 값을 추적하는 Grade 클래스가 필요하다.

In [14]:
class Grade(object):
    def __init__(self):
        self._values = {}
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        
        return self._value.get(instance, 0)
    
    def __set(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        
        self._values[instance] = value

* 하지만 이 코드는 **메모리 누수**가 발생할 수 있음
  * 프로그램 수명 동안 `__set__`에 전달된 모든 Exam 인스턴스의 참조를 저장함
  * 가비지 컬렉터가 정리하지 못하게 됨

* 파이썬 내장 모듈인 `weakref`를 사용하면 해결할 수 있다.
  * `_value` 딕셔너리 대신 `WeakKeyDictionary` 라는 클래스를 사용

* `WeakKeyDictionary`
  * 런타임 중에 자신이 마지막으로 남은 인스턴스의 참조를 가지고 있다는 사실을 알면 키 집합에서 해당 인스턴스를 제거한다.
  * 모든 인스턴스가 더는 사용되지 않으면 딕셔너리가 비어 있게 될 것이다.

In [15]:
from weakref import WeakKeyDictionary

class Grade(object):
    def __init__(self):
        self._values = WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        
        return self._value.get(instance, 0)
    
    def __set(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        
        self._values[instance] = value

In [16]:
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [17]:
first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75

print('First ', first_exam.writing_grade, 'is right')
print('Second', second_exam.writing_grade, 'is right')

First  82 is right
Second 75 is right


### 핵심 정리
* 직접 디스크립터 클래스를 정의하여 `@property` 메서드의 동작과 검증을 재사용하자
* `WeakKeyDictionary` 를 사용하여 디스크립터 클래스가 메모리 누수를 일으키지 않게 하자
* `__getattribute__` 가 디스크립터 프로토콜을 사용하여 속성을 얻어오고 설정하는 원리를 정확히 이해하려는 함정에 빠지지 말자 (???? Better Way 32 참조?)